# 출처 : https://velog.io/@kjh1337/%EB%84%A4%EC%9D%B4%EB%B2%84-%EC%87%BC%ED%95%91%EB%AA%B0-%EB%A6%AC%EB%B7%B0-%ED%81%AC%EB%A1%A4%EB%A7%81

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import requests
import re
import pandas as pd
import numpy as np
import os

from selenium.webdriver.common.keys import Keys

import warnings
warnings.filterwarnings('ignore')

In [2]:
name=['보다나 트리플 플로우 물결 고데기 크리미블루 40형']
category=['별점']

#address
ns_address="https://search.shopping.naver.com/catalog/28640667554?query=%EB%B3%B4%EB%8B%A4%EB%82%98%20%ED%8A%B8%EB%A6%AC%ED%94%8C%20%ED%94%8C%EB%A1%9C%EC%9A%B0%20%EB%AC%BC%EA%B2%B0%20%EA%B3%A0%EB%8D%B0%EA%B8%B0%20%ED%81%AC%EB%A6%AC%EB%AF%B8%EB%B8%94%EB%A3%A8%2040%ED%98%95&NaPm=ct%3Dl4i2oz3s%7Cci%3D2f3c84ac58c9161866b5a0ef7228d739e6b72414%7Ctr%3Dslsl%7Csn%3D95694%7Chk%3D9bd6e369589d5ca6dc17f187c8b39f2c57a36cb9"

#xpath
shoppingmall_review="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul"

In [9]:
header = {'User-Agent': ''}
d = webdriver.Chrome('C:/Users/admin/Desktop/chromedriver_win32/chromedriver.exe') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address)
req = requests.get(ns_address,verify=False)
html = req.text 
soup = BeautifulSoup(html, "html.parser")
sleep(2)

#쇼핑몰 리뷰 보기
d.find_element_by_xpath(shoppingmall_review).click()
sleep(2)

element=d.find_element_by_xpath(shoppingmall_review)
d.execute_script("arguments[0].click();", element)
sleep(2)

# 리뷰 가져오기
d.find_element_by_xpath(shoppingmall_review).click() #스크롤 건드리면 안됨
name_=name[0]
category_=category[0]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1

def add_dataframe(name,category,reviews,stars,cnt):  #데이터 프레임에 저장
    #데이터 프레임생성
    df1=pd.DataFrame(columns=['type','category','review','star'])
    n=1
    if (cnt>0):
        for i in range(0,cnt-1):
            df1.loc[n]=[name,category,reviews[i],stars[i]] #해당 행에 저장
            i+=1
            n+=1
    else:
        df1.loc[n]=[name,category,'null','null']
        n+=1    
    return df1

while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li[1]/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<5:#page10
        try: #리뷰의 마지막 페이지에서 error발생
            page +=1
            next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click() 
            
        except: break #리뷰의 마지막 페이지에서 process 종료
            
    elif page == 5 :
        break
        
    else : 
        try: #page11부터
            page+=1
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a[11]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page%10+2)+']').click()

        except: break
            
d.quit()           

df4=add_dataframe(name_,category_,reviews,stars,cnt)

페이지 1 

1 보다나 고데기 신세계네요. 머리가 깔
보다나 고데기 신세계네요. 머리가 깔끔하게 정돈 되면서 굵은 웨이브라서 자연스러워요
처음 사용하는거라서 조심스러워서 강하게 안하고 살짝씩만 찝어줘 봤는데 머릿결 손상도 없고 굉장히 자연스러운 연출을 할 수 있어서 좋았답니다.
고데기 하고 나서 웨이브 유지되는 시간도 길어서 금방 풀리거나 하지 않는 점이 맘에 들어요
여러 굵기중에 이 굵기를 선택하기를 잘 한거 같아요
너무 촘촘한 웨이브는 부담스러워서 자연스러운 물결머리를 원했거든요
웨이브를 더 굵게 하고 싶으면 고데기 집었던 간견을 멀리 띄우면 더 굵게 되기도 하고 간견을 바짝 붙히면 좀더 촘촘한 웨이브도 연출 할 수 있더라구요
저처럼 머리 잘 못 만지는 빠르고 자연스러운 연출을 할 수 있는점이 좋습니다
분들은 훨씬 더 예쁠거예요
고데기 웨이브 잡는 앞쪽이 두껍다 보니 좀 무게가 있긴하지만 쓸만해요
웨이브 위쪽은 뜨거우니까 조심하세요
장갑이랑 머리집게도 들어 있어서
저는 고데기를 나란히 평면으로 집어줘서 머리결이 전체적으로 같은 물결이 동일하게 나왔지만 고데기를 비스듬하게 해서 열을 가해주면 훨씬더 자연스러운 웨이브를 만들 수도 있네요
사진의 ㅡ 노란머리는 고데기로 가볍게 열을 가열해서 바로 찍어 본 사진이에요
사진의 ㅡ 검정머리는 정식으로 한건 아니고 전날 저녁에 시범 삼아 가볍게 고데기로 집어줘 봤는데, 하루 자고나면 다 풀릴 줄 알았는데 안풀려서 그대로 외출했어요
이사진은 다음날 오후사진이구요. 오른쪽 머리쪽은 아주 살짝 댔다가 뗐더니 좀 약하게 나왔어요.
고데기 한 다음날 저녁에 머리 감기 전까지 약간풀리긴 했지만, 웨이브가 잘 유지가 되더라구요
당일 웨이브는 진짜 잘 안풀리고 유지됐어요
단, 비오는날 습도 높은날은 풀리겠죠
고데기 하기전에 에센스 발라서 머릿결 보호 하고 사용하세요. 그럼 머리결어 윤기가 흐르네요
고데시 감은머리를 덜 말리고 사용하면 머릿결 상하는건 다 아시죠? 잘 말리시고 사용하세요^^ 평점5 

2 평소 보다나 덕후라 이번에 보

28 좋아요
뭐랄까 중간중간 연기가 나는데 기계이상인지 원래 그럴수도 있는지 모르겠네요.. 그리고 제 오른쪽은 물결 잘나오는데 완쪽은 왜 웨이브가 약한지 모르겠어요.. 두번 써봤는데 두번다 그러네요.. 연기에 대해서 문의해보니 일단 보내라고는 하셨는데 업체확인해서 이상없는거면 택배비 물어야헌단 얘기가 좀… ㅎㅎ 분명 원래 그런건지 이상한건지를 물어본건데 평점4 

29 배송도 빨리 해주시고 좋은 가격에 열
배송도 빨리 해주시고 좋은 가격에 열방지 장갑도 있어서 넘 좋았어요~ 익숙하지 않다보니 장갑이 너무 유용하더라구요~ 도착하자마자 사용해봤는데 사용법도 편하고 좋았네요~ 자연스러운 물결컬이 완성되었네요~ 익숙해지면 더 이쁘게 나올거 같아요 ㅎㅎ 영한 느낌의 웨이브가 너무 마음에 드네요 ㅎㅎ 평점4 

30 집에오자마자 풀러봤네요 ㅎㅎ 무게감은 봉고데기보다 좀 있지만 그래도 확실히 판고데기보다 물결도 잘 말아지고 머릿결도 덜 상하는거 같아서 만족합니다! 길고굵은모발이라 40m 딱 좋고
집에오자마자 풀러봤네요 ㅎㅎ 무게감은 봉고데기보다 좀 있지만 그래도 확실히 판고데기보다 물결도 잘 말아지고 머릿결도 덜 상하는거 같아서 만족합니다! 길고굵은모발이라 40m 딱 좋고 물결파마 좋아하는데 이젠 파마걱정 안해도 되겠어요 ㅎㅎ 평점4 

31 엄청난 고민고민 끝에 구입정말 잘한거 같아요정말정말 똥손이라 구입 고민 많았거든요유투브도 많이보고 몇번하니 금방 잘 돼요무엇보다 컬이 넘 이뿌게 나와 좋아요배송문의에도 친절히 응해
엄청난 고민고민 끝에 구입정말 잘한거 같아요
정말정말 똥손이라 구입 고민 많았거든요
유투브도 많이보고 몇번하니 금방 잘 돼요
무엇보다 컬이 넘 이뿌게 나와 좋아요
배송문의에도 친절히 응해주시구 넘 기분 좋은 구입이예요 평점4 

32 유튜브 보고 주문을 해봤어용 ㅎㅎ 곰손도 가능하다길래.. 그런데 생각보다 무거운건 아닌것 같고 괜찮아요. 다만 오랫동안 하게되면 힘들긴 합니다 ㅎ 섹션도 잘 나눠서 하면 정말 잘
유튜브 보고 주문을 해봤어용 ㅎㅎ 곰손도 가능하

75 굵은 히피펌 느낌도 낼 수 있고 생각보다 무겁지 않던데요 전,,, 힘이 센건가 ㅋㅋ 아무튼 손 자주 갈 것 같네용
굵은 히피펌 느낌도 낼 수 있고 생각보다 무겁지 않던데요 전,,, 힘이 센건가 ㅋㅋ 아무튼 손 자주 갈 것 같네용 평점2 

76 기술 없이도 물결펌 잘 나와서 좋아요! 다만 전원 아래 이음부분에 살이 잘 찝혀서 손가락에 멍이 들었어요ㅠ
기술 없이도 물결펌 잘 나와서 좋아요! 다만 전원 아래 이음부분에 살이 잘 찝혀서 손가락에 멍이 들었어요ㅠ 평점2 

77 다들 쉽다고 했는데 저는 좀 적응하기 힘들었어요ㅋㅋ 근데 이제 손에 익히니까 물결 너무 예쁘게나와요☺️
다들 쉽다고 했는데 저는 좀 적응하기 힘들었어요ㅋㅋ 근데 이제 손에 익히니까 물결 너무 예쁘게나와요☺️ 평점2 

78 아직 사용해보진 않았지만 기대됩니다~!!색상도 마음에 들고 실리콘도 와서 좋은 거 같아요 ㅎㅎ똥손에게는 꿀템이 될 것 같습니다~!
아직 사용해보진 않았지만 기대됩니다~!!
색상도 마음에 들고 실리콘도 와서 좋은 거 같아요 ㅎㅎ
똥손에게는 꿀템이 될 것 같습니다~! 평점2 

79 오자마자 해봤는데 후기처럼 무게감이 조금 있습니다 워낙 똥손이라 몇번 더 해봐야 감을 잡을 것 같아요;)
오자마자 해봤는데 후기처럼 무게감이 조금 있습니다 워낙 똥손이라 몇번 더 해봐야 감을 잡을 것 같아요;) 평점2 

80 계속 쓰다보니 노하우가 생겨서 전날 빡세게 찝고 자면 다음날 자연스러운 물결헤어로 다닐 수 있어용 굳!
계속 쓰다보니 노하우가 생겨서 전날 빡세게 찝고 자면 다음날 자연스러운 물결헤어로 다닐 수 있어용 굳! 평점2 

페이지 5 

81 물결펌 너무 하고싶은데 미용실가서하면 원하는컬 안나오고ㅠㅠ 고민고민하다 샀는데 만족스러워용 무거운건 어쩔수없지만ㅠ
물결펌 너무 하고싶은데 미용실가서하면 원하는컬 안나오고ㅠㅠ 고민고민하다 샀는데 만족스러워용 무거운건 어쩔수없지만ㅠ 평점5 

82 조작법도 쉽고 컬도 잘나와서 만족합니당다만 온도조절 버튼이 이전 보다나와 다르게 버튼식이라머리

In [10]:
df4

,type,category,review,star
1,보다나 트리플 플로우 물결 고데기 크리미블루 40형,별점,보다나 고데기 신세계네요. 머리가 깔\n보다나 고데기 신세계네요. 머리가 깔끔하게 ...,평점5
2,보다나 트리플 플로우 물결 고데기 크리미블루 40형,별점,평소 보다나 덕후라 이번에 보다나에서 물결고데기가 나왔다는 소식을 듣고 얼른~! 4...,평점5
3,보다나 트리플 플로우 물결 고데기 크리미블루 40형,별점,"고데기는 전부 보다나제품으로 가지고있어요판고데기,봉고데기36mm 로 가지고있는데 물...",평점5
4,보다나 트리플 플로우 물결 고데기 크리미블루 40형,별점,한달 후기를 쓰자면제가 기존에 산 스프링 컬렉션 >초록색은지금 나온 파란색보다 무거...,평점5
5,보다나 트리플 플로우 물결 고데기 크리미블루 40형,별점,예전부터 갖고싶어서 눈여겨보다 보통 고데기에 비해 가격이 좀 부담되어 미루다 이번에...,평점5
...,...,...,...,...
96,보다나 트리플 플로우 물결 고데기 크리미블루 40형,별점,처음엔 좀 어색했는데 두번째부터는 잘하고있어요~~ 예쁘게 물결웨이브 쉽게하게되어서 ...,평점5
97,보다나 트리플 플로우 물결 고데기 크리미블루 40형,별점,평이좋아서구매해봤는데색상이나배송도빠르고넘맘에드네여 ㅎ똥손이라서잘사용할거같아여 ㅎ\n...,평점5
98,보다나 트리플 플로우 물결 고데기 크리미블루 40형,별점,진짜 쉬워요. 긴머린데 다 하는데 10분정도 걸리고 요령 없어도 잘 되던데 저는 너...,평점5
99,보다나 트리플 플로우 물결 고데기 크리미블루 40형,별점,제가 똥손이라그런지 모양이예쁘게안나오네요 ㅠㅠ 연습좀해봐야겠습니다.. 제품자체는좋아...,평점5


# # csv로 저장

In [12]:
df4.to_csv("./naverReviews.csv", header=True, index=True)